<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/PVGIS_TMY_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install requests pandas

In [33]:
import pandas as pd
import requests
import io
import time

cities = {
    "Turku": {"lat": 60.45, "lon": 22.26},
    "Copenhagen": {"lat": 55.67, "lon": 12.56},
    "Stockholm": {"lat": 59.32, "lon": 18.06},
    "Oslo": {"lat": 59.91, "lon": 10.75}
}

# The endpoint for TMY is different from Series Calculation
base_url = "https://re.jrc.ec.europa.eu/api/v5_3/tmy"

for city, coords in cities.items():
    params = {
        'lat': coords['lat'],
        'lon': coords['lon'],
        'outputformat': 'csv'
    }

    try:
        print(f"Downloading TMY data for {city}...")
        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"❌ Server error for {city}. Code: {response.status_code}")
            continue

        raw_text = response.text
        lines = raw_text.splitlines()

        # TMY header starts with "time(UTC)"
        header_index = next((i for i, line in enumerate(lines) if "time(UTC)" in line), -1)

        if header_index == -1:
            print(f"❌ TMY Header not found for {city}.")
            continue

        # Extract the 8760 hours + header
        data_lines = lines[header_index : header_index + 8761]
        df = pd.read_csv(io.StringIO("\n".join(data_lines)))

        # --- COLUMN MAPPING FOR TMY ---
        # TMY uses different labels:
        # G(h) is global horizontal irradiance
        # T2m is dry bulb temp
        # WS10m is wind speed

        # 1. Format Time
        # TMY format is usually 'MM-DD HH:MM' or similar. We'll simplify it.
        df['dt_obj'] = pd.to_datetime(df['time(UTC)'], format='%m-%d %H:%M', errors='coerce')
        df['Month-Day'] = df['dt_obj'].dt.strftime('%m-%d')
        df['Hour'] = df['dt_obj'].dt.strftime('%H:%M')

        # 2. Select and Rename
        # We look for G(h) for historical average GHI
        df_filtered = df[['Month-Day', 'Hour', 'G(h)', 'WS10m', 'T2m']].copy()
        df_filtered.columns = ['Month-Day', 'Hour', 'Avg GHI (W/m2)', 'Avg Wind Speed (m/s)', 'Avg Temp (C)']

        # 3. Save
        filename = f"{city.lower()}_historical_tmy.csv"
        df_filtered.to_csv(filename, index=False)
        print(f"✅ Successfully saved TMY: {filename}")

        time.sleep(2)

    except Exception as e:
        print(f"❌ Error processing {city}: {e}")

print("\nProcess finished. You now have the historical average files.")

✅ Successfully saved TMY: turku_historical_tmy.csv
✅ Successfully saved TMY: copenhagen_historical_tmy.csv
✅ Successfully saved TMY: stockholm_historical_tmy.csv
✅ Successfully saved TMY: oslo_historical_tmy.csv

Process finished. You now have the historical average files.
